In [1]:
#! /usr/bin/env python
"""
Compute elevation statistics for the debris-covered areas in each latitude and longitude
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms

In [2]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

print('All DC glaciers:', dc_shp.shape[0], 'All DC Area (km2):', dc_shp.DC_Area_v2.sum() / 1e6)

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)

rgino_str_list_subset = [x.split('-')[1] for x in dc_shp_subset.RGIId.values]

print('Subset DC glaciers:', dc_shp_subset.shape[0], 'Subset DC Area (km2):', dc_shp_subset.DC_Area_v2.sum() / 1e6)

dc_shp_subset

All DC glaciers: 6074 All DC Area (km2): 1889.725456
Subset DC glaciers: 598 Subset DC Area (km2): 1130.456647


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Name,DC_Area,DC_BgnDate,DC_EndDate,DC_CTSmean,DC_Area_%,area_singl,DC_Area_v2,DC_Area__1,geometry
0,RGI60-17.00023,G289839E34567S,20009999,20030531,-70.160172,-34.566343,17,2,2.893,3346,...,None,244800,2013,2017,46.558741,8.462,130569,175592,6.070,"MULTIPOLYGON (((-70.15492 -34.56166, -70.15427..."
1,RGI60-17.00080,G289673E34997S,20009999,20030531,-70.328539,-34.998345,17,2,8.805,2989,...,None,1340100,2013,2017,46.167707,15.220,55825,1015420,11.532,"MULTIPOLYGON (((-70.35320 -35.01028, -70.35090..."
2,RGI60-17.00141,G286990E50944S,20009999,20030531,-73.016432,-50.951110,17,1,2.254,909,...,None,1596600,2013,2017,24.138179,70.834,27009,1587205,70.417,"MULTIPOLYGON (((-73.00803 -50.95680, -73.00632..."
3,RGI60-17.00172,G286530E49887S,20009999,20030531,-73.221800,-49.696863,17,1,838.044,193,...,Upsala + Cono,24186600,2013,2017,29.969424,2.886,25210,23455993,2.799,"MULTIPOLYGON (((-73.42634 -49.92168, -73.42593..."
4,RGI60-17.00466,G289424E35275S,20009999,20030531,-70.578001,-35.268075,17,2,13.681,2937,...,None,1649700,2013,2017,68.794855,12.058,251626,1213447,8.870,"MULTIPOLYGON (((-70.58397 -35.29799, -70.58364..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,RGI60-17.15898,G286431E47184S,20011103,-9999999,-73.725850,-47.262163,17,1,268.749,28,...,Acodado,6780600,2013,2017,11.617462,2.523,34221,6129660,2.281,"MULTIPOLYGON (((-73.82172 -47.38723, -73.82171..."
594,RGI60-17.15899,G286284E47139S,20011103,-9999999,-73.741838,-47.161817,17,1,153.237,50,...,HPN 1,11167200,2013,2017,10.497445,7.288,24314,10001167,6.527,"MULTIPOLYGON (((-73.76910 -47.25683, -73.76831..."
595,RGI60-17.15903,G286330E47046S,20011103,-9999999,-73.716944,-47.050215,17,1,161.389,16,...,Benito,5054400,2013,2017,12.873844,3.132,131481,4471130,2.770,"MULTIPOLYGON (((-73.71341 -47.08398, -73.71301..."
596,RGI60-17.15904,G286335E46976S,20011103,-9999999,-73.687680,-46.976967,17,1,31.033,68,...,Fraenkel,1294200,2013,2017,10.791264,4.170,173798,1168863,3.767,"MULTIPOLYGON (((-73.71390 -46.98113, -73.71351..."


In [3]:
# Select glaciers using RGI and find unique latlons
#  (Scherler DC shapefiles do not have same CenLat and CenLon for some reason)
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgino_str_list_subset)
main_glac_rgi_subset['CenLon_360'] = main_glac_rgi_subset['CenLon']
main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'])

# Load met data and find nearest latlon indices
ds = xr.open_dataset(debris_prms.metdata_fp + '../' + debris_prms.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi_subset['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi_subset['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))
latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
main_glac_rgi_subset['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_subset['latlon_unique_no'] = main_glac_rgi_subset['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_subset['latlon_unique_no'])), '\n\n')

# Delete me
latlon_nearidx_unique_v1 = latlon_nearidx_unique.copy()

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

598 glaciers in region 17 are included in this model run: ['00023', '00080', '00141', '00172', '00466', '00516', '01019', '01100', '01101', '01108', '01112', '01123', '01128', '01218', '01236', '01253', '01421', '01429', '01448', '01495', '01506', '01554', '01638', '01687', '01762', '01879', '01917', '01921', '01940', '01970', '01972', '01974', '02025', '02035', '02071', '02265', '02427', '02504', '02519', '02531', '02547', '02600', '02617', '02642', '02665', '02666', '02671', '02674', '02685', '02687'] and more
This study is focusing on 598 glaciers in region [17]
unique lat/lons: 219 




In [4]:
# ===== LOAD GLACIERS WITH DATA =====
main_glac_rgi_subset['mb_fn'] = np.nan
mb_binned_fp = debris_prms.mb_binned_fp

regions_str = [str(x).zfill(2) for x in debris_prms.roi_rgidict[debris_prms.roi]]

mb_fns = []
mb_rgiids = []
for i in os.listdir(mb_binned_fp):
    if i.endswith('_mb_bins.csv') and i.split('_')[0].split('.')[0].zfill(2) in regions_str:
        mb_fns.append(i)
        rgiid_raw = i.split('_')[0]
        rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
        mb_rgiids.append(rgiid)
mb_rgiids = sorted(mb_rgiids)
mb_fns = sorted(mb_fns)
mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
mb_fn_df['RGIId'] = mb_rgiids
mb_fn_df['mb_fn'] = mb_fns

# Find glaciers that are debris-covered
mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) if value in list(main_glac_rgi_subset.RGIId.values)]
mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')

print('Debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')

mb_fn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fn'].values))

main_glac_rgi_subset['mb_fn'] = main_glac_rgi_subset.RGIId.map(mb_fn_dict)

Debris-covered glaciers: 446 




In [5]:
# ===== SELECT GLACIERS WITH DATA ====
main_glac_rgi_wobs = main_glac_rgi_subset.dropna(subset=['mb_fn']).copy()
# print('subset wdata length:', main_glac_rgi_wobs.shape)
main_glac_rgi_wobs.reset_index(inplace=True, drop=True)

# Update the latlon unique pickle files
latlon_nearidx_unique = sorted(list(set(main_glac_rgi_wobs['latlon_nearidx'].values)))
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_wobs['latlon_unique_no'] = main_glac_rgi_wobs['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_wobs['latlon_unique_no'])), '\n\n')

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)
    
main_glac_rgi_wobs

unique lat/lons: 166 




,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,latlon_nearidx,latlon_unique_no,mb_fn
0,171,RGI60-17.00172,-73.4697,-49.8866,17,1,838.044,193,3029,1347,...,2,9,20019999,172,17.00172,17.00172,286.5303,"(560, 1146)",108,17.00172_mb_bins.csv
1,515,RGI60-17.00516,-70.1061,-34.2829,17,2,6.185,2862,4446,3474,...,0,9,20019999,516,17.00516,17.00516,289.8939,"(497, 1160)",16,17.00516_mb_bins.csv
2,1018,RGI60-17.01019,-73.4487,-51.4220,17,1,12.542,131,1928,1051,...,0,9,20019999,1019,17.01019,17.01019,286.5513,"(566, 1146)",121,17.01019_mb_bins.csv
3,1235,RGI60-17.01236,-70.1570,-34.5824,17,2,2.039,3185,3874,3526,...,0,9,20019999,1236,17.01236,17.01236,289.8430,"(498, 1159)",17,17.01236_mb_bins.csv
4,1252,RGI60-17.01253,-70.0365,-34.4397,17,2,4.655,3375,4318,3877,...,0,9,20019999,1253,17.01253,17.01253,289.9635,"(498, 1160)",18,17.01253_mb_bins.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,15897,RGI60-17.15898,-73.5690,-47.1840,17,1,268.749,28,3277,1256,...,2,9,20011103,15898,17.15898,17.15898,286.4310,"(549, 1146)",61,17.15898_mb_bins.csv
442,15898,RGI60-17.15899,-73.7160,-47.1390,17,1,153.237,50,2063,985,...,0,9,20011103,15899,17.15899,17.15899,286.2840,"(549, 1145)",60,17.15899_mb_bins.csv
443,15902,RGI60-17.15903,-73.6700,-47.0460,17,1,161.389,16,2401,945,...,2,9,20011103,15903,17.15903,17.15903,286.3300,"(548, 1145)",57,17.15903_mb_bins.csv
444,15903,RGI60-17.15904,-73.6650,-46.9760,17,1,31.033,68,2192,1274,...,2,9,20011103,15904,17.15904,17.15904,286.3350,"(548, 1145)",57,17.15904_mb_bins.csv


In [6]:
# print('DELETE ME - HACK FOR DEVELOPMENT')
# print(np.where(main_glac_rgi_wobs['latlon_unique_no'] == 172)[0])
# main_glac_rgi_wobs = main_glac_rgi_wobs.loc[372:373,:]
# main_glac_rgi_wobs['mb_fn'].values

In [7]:
# ===== DEBRIS ELEVATION STATS ====================================================================================
# CALCULATE DEBRIS ELEVATION STATS FOR GLACIERS WITH DATA FOR EACH UNIQUE LAT/LON
elev_stats_latlon_dict = {}
latlon_list_updated = []
rgiid_4cal = []

for nlatlon, latlon_unique in enumerate(np.unique(main_glac_rgi_wobs.latlon_unique_no)):
# for nlatlon, latlon_unique in enumerate([np.unique(main_glac_rgi_wobs.latlon_unique_no)[0]]):

    main_glac_rgi_subset = main_glac_rgi_wobs[main_glac_rgi_wobs['latlon_unique_no'] == latlon_unique]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    # Debris elevation stats should be done by lat/lon
    df_all = None
    elev_list_all = []
    df_idx_count = 0
    count_width_passes = 0
    for nglac, glac_fn in enumerate(main_glac_rgi_subset.mb_fn.values):
#     for nglac, glac_fn in enumerate([main_glac_rgi_subset.mb_fn.values[0]]):

        glac_fullfn = debris_prms.mb_binned_fp + glac_fn
        
        glac_str_noleadzero = glac_fullfn.split('/')[-1].split('_')[0]
        rgiid = 'RGI60-' + glac_str_noleadzero.split('.')[0].zfill(2) + '.' + glac_str_noleadzero.split('.')[1]

        assert main_glac_rgi_subset.loc[nglac,'RGIId'] == rgiid, 'RGIId does not matach mass balance filename'
        
        # Select bins that meet calibratioin criteria
        df_raw = pd.read_csv(glac_fullfn)
        df = df_raw.dropna(subset=['mb_bin_mean_mwea'])
        df['z1_bin_areas_perc_cum'] = np.cumsum(df['z1_bin_area_valid_km2']) /df['z1_bin_area_valid_km2'].sum() * 100
        # add width to bins
        widths_fp = debris_prms.oggm_fp + 'widths/' + 'RGI60-' + rgiid.split('-')[1].split('.')[0] + '/'
        widths_fn = rgiid + '_widths_m.csv'
        try:
            # Add width to each elevation bin
            widths_df = pd.read_csv(widths_fp + widths_fn)
            elev_nearidx = (np.abs(df['bin_center_elev_m'].values[:,np.newaxis] - 
                                   widths_df['elev'].values).argmin(axis=1))
            df['width_m'] = widths_df.loc[elev_nearidx,'width_m'].values
        except:
            df['width_m'] = 0
        
        df_idx = np.where((df['vm_med'] <= debris_prms.vel_threshold) 
                          & (df['width_m'] >= debris_prms.width_min_dict[debris_prms.roi])
                          & (df['dc_bin_area_perc'] >= debris_prms.debrisperc_threshold)
                          & (df['dc_bin_count_valid'] >= 10)
                          & (df['z1_bin_areas_perc_cum'] <= debris_prms.term_area_perc)
                          )[0]
        df_debris = df.loc[df_idx,:]
        df_debris.reset_index(inplace=True, drop=True)
        df_idx_count += len(df_idx)
        
            
        if len(df_idx) > 0:
            for nelev, elev in enumerate(list(df_debris['bin_center_elev_m'].values)):
                elev_list_single = list(np.repeat(elev, df_debris.loc[nelev,'dc_bin_count_valid']))
                elev_list_all.extend(elev_list_single)
            
#             # only work with terminus
#             df_idx_dif = list(df_idx[1:] - df_idx[:-1])
#             if np.sum(df_idx_dif) == len(df_idx)-1:
#                 df_idx_nojump = df_idx
#             else:
#                 idx_jumpinbins = df_idx_dif.index(next(filter(lambda x: x>1, df_idx_dif)))
#                 df_idx_nojump = df_idx[0:idx_jumpinbins+1]
#             df_debris_nojump = df_debris.loc[df_idx_nojump,:]
#             df_debris_nojump.reset_index(inplace=True, drop=True)
#             # Median width to ensure terminus velocities can be estimated
#             width_median = np.median(widths_m[np.where(h < df_debris_nojump['bin_center_elev_m'].max())[0]])
#             if width_median > debris_prms.width_min_dict[debris_prms.roi]:
#                 for nelev, elev in enumerate(list(df_debris_nojump['bin_center_elev_m'].values)):
#                     elev_list_single = list(np.repeat(elev, df_debris_nojump.loc[nelev,'dc_bin_count_valid']))
#                     elev_list_all.extend(elev_list_single)
#                 count_width_passes += 1
    
            rgiid_4cal.append(rgiid.split('-')[1])
        
    if df_idx_count > 0:
        dc_zmean = np.mean(elev_list_all)
        dc_zstd = np.std(elev_list_all)
        dc_zmed = malib.fast_median(elev_list_all)
        dc_zmad = malib.mad(elev_list_all)
        
        lat_deg = float(ds.latitude[latlon_unique_dict_reversed[latlon_unique][0]].values)
        lon_deg = float(ds.longitude[latlon_unique_dict_reversed[latlon_unique][1]].values)
        elev_stats_latlon_dict[lat_deg,lon_deg] = [dc_zmean, dc_zstd, dc_zmed, dc_zmad]
        latlon_list_updated.append((lat_deg, lon_deg))
        
print('unique lat/lons updated:', len(latlon_list_updated))
# Update pickle of unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list_updated, f)

unique lat/lons updated: 122


In [8]:
# Statistics of data coverage
rgiid_4cal = sorted(rgiid_4cal)
main_glac_rgi_4cal = debris_prms.selectglaciersrgitable(rgiid_4cal)
dc_area_dict = dict(zip(dc_shp.RGIId.values, dc_shp.DC_Area_v2.values))
main_glac_rgi_4cal['DC_Area_v2'] = main_glac_rgi_4cal.RGIId.map(dc_area_dict)
print('\nDC glaciers (used for cal):', main_glac_rgi_4cal.shape[0], 
      'DC Area (used for cal, km2):', main_glac_rgi_4cal.DC_Area_v2.sum() / 1e6)

284 glaciers in region 17 are included in this model run: ['00516', '01019', '01448', '01506', '01687', '01917', '01940', '01970', '01972', '02025', '02071', '02427', '02531', '02600', '02642', '02666', '02674', '02697', '02698', '02734', '02737', '02806', '02857', '03229', '03289', '03291', '03292', '03301', '03302', '03307', '03309', '03311', '03318', '03319', '03325', '03348', '03353', '03359', '03371', '03382', '03406', '03418', '03424', '03425', '03451', '03455', '03469', '03519', '03531', '03545'] and more
This study is focusing on 284 glaciers in region [17]

DC glaciers (used for cal): 284 DC Area (used for cal, km2): 553.224648


In [9]:
# ===== ADD DEBRIS ELEVATION STATS TO MET DATA ======
overwrite_dc_stats = True
for nlatlon, latlon in enumerate(latlon_list_updated):
# for nlatlon, latlon in enumerate([latlon_list_updated[0]]):
    
    lat_deg = latlon[0]
    lon_deg = latlon[1]
    
    print(nlatlon, lat_deg, lon_deg)
    
    if lat_deg < 0:
        lat_str = 'S-'
    else:
        lat_str = 'N-' 

    # ===== Meteorological data =====
    metdata_fn = debris_prms.metdata_fn_sample.replace(
        'XXXX', str(int(np.abs(lat_deg)*100)) + lat_str + str(int(lon_deg*100)) + 'E-')
    
    ds = xr.open_dataset(debris_prms.metdata_fp + metdata_fn) 
    try:
        print('  existed:', ds.dc_zmean.values, 'vs', elev_stats_latlon_dict[latlon][0])
    except:
        pass
    if 'dc_zmean' not in list(ds.keys()) or overwrite_dc_stats:
        # Add stats
        ds['dc_zmean'] = elev_stats_latlon_dict[latlon][0]
        ds['dc_zmean'].attrs = {'units':'m a.s.l.', 'long_name':'Mean debris cover elevation', 
                                'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zstd'] = elev_stats_latlon_dict[latlon][1]
        ds['dc_zstd'].attrs = {'units':'m a.s.l.', 'long_name':'Standard deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmed'] = elev_stats_latlon_dict[latlon][2]
        ds['dc_zmed'].attrs = {'units':'m a.s.l.', 'long_name':'Median debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmad'] = elev_stats_latlon_dict[latlon][3]
        ds['dc_zmad'].attrs = {'units':'m a.s.l.', 'long_name':'Median absolute deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}

        try:
            ds.close()
        except:
            continue
            
        # Export updated dataset
        ds.to_netcdf(debris_prms.metdata_fp + metdata_fn, mode='a')
    else:
        print(lat_deg, lon_deg, 'exists')

0 -29.25 290.0
  existed: 4819.727891156463 vs 4819.727891156463
1 -32.0 289.75
  existed: 4250.27446300716 vs 4250.27446300716
2 -32.0 290.0
  existed: 4244.733275032794 vs 4244.733275032794
3 -32.5 289.75
  existed: 4203.882978723404 vs 4203.882978723404
4 -32.5 290.0
  existed: 4154.938524590164 vs 4154.938524590164
5 -32.75 290.0
  existed: 4012.427681352915 vs 4012.427681352915
6 -33.0 290.0
  existed: 3496.3438368860056 vs 3496.3438368860056
7 -33.25 289.75
  existed: 4227.171945701358 vs 4227.171945701358
8 -33.25 290.0
  existed: 3506.244167962675 vs 3506.244167962675
9 -33.25 290.25
  existed: 4096.485411140584 vs 4096.485411140584
10 -33.5 290.0
  existed: 3464.79274611399 vs 3464.79274611399
11 -33.5 290.25
  existed: 3883.243875584916 vs 3883.243875584916
12 -33.75 290.0
  existed: 3420.9251680795087 vs 3420.9251680795087
13 -33.75 290.25
  existed: 3843.235867446394 vs 3843.235867446394
14 -33.75 290.5
  existed: 4352.144670050761 vs 4352.144670050761
15 -34.25 290.0
  exi

In [10]:
print('DONE!')

DONE!
